In [1]:
import pandas as pd
import json, re

def convert_coord(c):
    c = str(c)
    j = len(c) - 6
    d = int(c[0:2 + j])
    m = int(c[2 + j:4 + j])
    s = float(c[4 + j:6 + j] + '.' + c[6 + j:])
    q = 1 if j == 0 else -1
    coord = round(q * (d + m / 60 + s / 3600), 6)
    
    return coord

def pprint(dict):
    print(json.dumps(dict, indent=2))

def comma_followed_by_number(s):
    for i, char in enumerate(s[:-1]):
        if char == ',' and s[i+1].isdigit():
            return True
    return False

def extract_table_section_from_file(section_header, filename, offset=0):
    offset *= 3
    section_header = '******* ' + section_header + ' *******'
    with open(filename, "r") as file:
        lines = file.readlines()

    extracted_lines = []
    inside_section = False
    end_marker_count = 0

    for line in lines:
        if section_header in line:
            inside_section = True
            extracted_lines.append(line)
            continue

        if inside_section:
            if end_marker_count > offset:
                extracted_lines.append(line)
            # Count lines that are mostly dashes
            if line.strip().startswith('---'):
                end_marker_count += 1
                if end_marker_count >= 3 + offset:
                    break

    return "".join(extracted_lines)

def remove_dash_lines(text):
    cleaned_lines = [
        line for line in text.splitlines()
        if not line.strip().startswith("---")
    ]
    return "\n".join(cleaned_lines)

def convert_pipe_text_to_csv(multi_line_text):
    csv_lines = []
    for line in multi_line_text.splitlines():
        if not line.strip():
            continue
        if '|' not in line:
            continue
        
        fields = [field.strip() for field in line.strip('|').split('|')]
        csv_line = '|'.join(fields)
        csv_lines.append(csv_line)

    return '\n'.join(csv_lines)

def csv_text_to_dataframe(csv_text):
    lines = [line.strip() for line in csv_text.strip().split('\n') if line.strip()]
    
    headers = [h.strip() for h in lines[0].split('|')]
    
    data = []
    for line in lines[1:]:
        fields = [f.strip() for f in line.split('|')]
        data.append(fields)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def read_adaptation_section(section_header, filename, offset=0):
    text = extract_table_section_from_file(section_header, filename, offset)
    text = remove_dash_lines(text)
    text = convert_pipe_text_to_csv(text)
    
    return csv_text_to_dataframe(text)

In [2]:
df = read_adaptation_section('FIX', 'mia_r10c_y7_10c.txt', offset=0)

df = df[(df['SIM Only'] == 'N')]
df = df[['Name', 'Short Name', 'Description']]

sp = {}

for index, row in df.iterrows():
    name = row['Name']
    short_name = row['Short Name']
    desc = row['Description']

    if 'DUMMY' in desc or 'AIRPORT' in desc:
        continue

    if ',' in desc:
        if not comma_followed_by_number(desc):
            continue
    
    if len(name) == 3:
        sp[name] = desc
    elif len(short_name) == 3:
        sp[short_name] = desc
        
# print(pprint(sp))

In [3]:
df = read_adaptation_section('FIXPAIRS_TCP', 'mia_r10c_y7_10c.txt', offset=0)
fixes = sorted(df['fix1'].unique().tolist())
for fix in fixes:
    if fix in sp:
        print(fix, sp[fix], sep='\t')

07X	FIX PR - OVR PARTS
ANE	FIX PR - ANE PSEUDO FIX
ANJ	ANNEY INTERSECTION JETS
ANL	FIX PR - ANL PSEUDO FIX
ANQ	ANNEY INTERSECTION TURBO
ANR	FIX PR - ANR PSEUDO FIX
BG1	FIX PR - ARR PARTS
BG2	FIX PR - BG1 REASSIGN
BG3	FIX PR - BG1 REASSIGN
BHH	BHHIA
BLQ	BLUFI INTERSECTION TURBOS
CD1	FIX PR - CUD REASSIGN
CD2	FIX PR - CUD REASSIGN
CD3	FIX PR - CUD REASSIGN
CST	CSTAL
CU1	FIX PR - CUUDA NON-RNAV
CUD	FIX PR - ARR PARTS
CUR	CUURT
CUU	CUUDA
DEK	DEKAL INTERSECTION
DEP	FIX PR - DEKAL INTERSECTION
DK1	FIX PR - DEKAL FLL JET
DKL	FIX PR - DEKAL (METROPLEX)
DMY	FIX PR - OVR PARTS
FAM	FAMIN INTERSECTION
FG1	FIX PR - FGZ REASSIGN
FG2	FIX PR - FGZ REASSIGN
FG3	FIX PR - FGZ REASSIGN
FGZ	FIX PR - ARR PARTS
FL1	FIX PR - FLX REASSIGN
FLO	FIX PR - ARR PARTS
FLX	FIX PR - ARR PARTS
FM1	FIX PR - FAMIN (DVALL) PROPS
FO1	FIX PR - FORTL CONV ARR PROPS
FOR	FORTL INTERSECTION
FRE	FIX PR - ARR PARTS
FRO	FROGZ
GIL	GILBI INTERSECTION
HEP	FIX PR - HEATT PROPS
HEQ	FIX PR - HEATT INTERSECTION
HSX	FIX PR - OVR PARTS
JUN	